In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd

In [ ]:
referendum_path = "/Users/macbookpro/Desktop/UPSaclay Courses/T2/datacamp-master/01_pandas/2024-assignment-pandas/data/referendum.csv"
regions_path = "/Users/macbookpro/Desktop/UPSaclay Courses/T2/datacamp-master/01_pandas/2024-assignment-pandas/data/regions.csv"
departments_path = "/Users/macbookpro/Desktop/UPSaclay Courses/T2/datacamp-master/01_pandas/2024-assignment-pandas/data/departments.csv"

def load_data():
    """Load data from the CSV files referundum/regions/departments."""
    referendum = pd.read_csv(referendum_path, sep=';')
    regions = pd.read_csv(regions_path, sep=',')
    departments = pd.read_csv(departments_path, sep=',')

    return referendum, regions, departments


def merge_regions_and_departments(regions, departments):
    """Merge regions and departments in one DataFrame.

    The columns in the final DataFrame should be:
    ['code_reg', 'name_reg', 'code_dep', 'name_dep']
    """

    merged = pd.merge(departments, regions, left_on='region_code', right_on='code', suffixes=('_dep', '_reg'))
    regions_and_departments = merged[['code_reg', 'name_reg', 'code_dep', 'name_dep']].rename(columns={
        'code_reg': 'code_reg', 
        'name_reg': 'name_reg',
        'code_dep': 'code_dep',
        'name_dep': 'name_dep'
    })

    for i in range(1, 10):
        regions_and_departments['code_dep'] = regions_and_departments['code_dep'].str.replace(f"0{i}", f"{i}")

    return regions_and_departments


def merge_referendum_and_areas(referendum, regions_and_departments):
    """Merge referendum and regions_and_departments in one DataFrame.

    You can drop the lines relative to DOM-TOM-COM departments, and the
    french living abroad.
    """
    referendum_and_areas = pd.merge(referendum, regions_and_departments,
                                    left_on='Department code',
                                    right_on='code_dep')

    return referendum_and_areas


def compute_referendum_result_by_regions(referendum_and_areas):
    """Return a table with the absolute count for each region.

    The return DataFrame should be indexed by `code_reg` and have columns:
    ['name_reg', 'Registered', 'Abstentions', 'Null', 'Choice A', 'Choice B']
    """

    referendum_result_by_regions = referendum_and_areas.groupby(['code_reg', 'name_reg']).agg(
        Registered=('Registered', 'sum'),
        Abstentions=('Abstentions', 'sum'),
        Null=('Null', 'sum'),
        Choice_A=('Choice A', 'sum'),
        Choice_B=('Choice B', 'sum')
        ).reset_index()
    referendum_result_by_regions = referendum_result_by_regions.rename(columns={'Choice_A': 'Choice A', 'Choice_B': 'Choice B'})

    # Setting 'code_reg' as the index
    referendum_result_by_regions = referendum_result_by_regions.set_index('code_reg')

    return referendum_result_by_regions

def plot_referendum_map(referendum_result_by_regions):
    """Plot a map with the results from the referendum.

    * Load the geographic data with geopandas from `regions.geojson`.
    * Merge these info into `referendum_result_by_regions`.
    * Use the method `GeoDataFrame.plot` to display the result map. The results
      should display the rate of 'Choice A' over all expressed ballots.
    * Return a gpd.GeoDataFrame with a column 'ratio' containing the results.
    """

    geo_data = gpd.read_file('data/regions.geojson')
    geo_data = geo_data.merge(referendum_result_by_regions, left_on='nom', right_on='name_reg')
    geo_data = geo_data.rename(columns={'Choice_A': 'Choice A', 'Choice_B': 'Choice B'})

    geo_data['ratio'] = geo_data['Choice A'] / (geo_data['Choice A'] + geo_data['Choice B'])
    geo_data.plot(column='ratio', legend=True, cmap='coolwarm')

    return geo_data

referendum, regions, departments = load_data()
regions_and_departments = merge_regions_and_departments(regions, departments)
referendum_and_areas = merge_referendum_and_areas(referendum, regions_and_departments)
referendum_result_by_regions = compute_referendum_result_by_regions(referendum_and_areas)
# plot_referendum_map(referendum_result_by_regions)

In [71]:
referendum

,Department code,Department name,Town code,Town name,Registered,Abstentions,Null,Choice A,Choice B
0,1,AIN,1,L'Abergement-Clémenciat,592,84,9,154,345
1,1,AIN,2,L'Abergement-de-Varey,215,36,5,66,108
2,1,AIN,4,Ambérieu-en-Bugey,8205,1698,126,2717,3664
3,1,AIN,5,Ambérieux-en-Dombes,1152,170,18,280,684
4,1,AIN,6,Ambléon,105,17,1,35,52
...,...,...,...,...,...,...,...,...,...
36786,ZZ,FRANCAIS DE L'ETRANGER,7,Europe centrale,89643,54981,318,17055,17289
36787,ZZ,FRANCAIS DE L'ETRANGER,8,"Europe du Sud, Turquie, Israël",109763,84466,292,9299,15706
36788,ZZ,FRANCAIS DE L'ETRANGER,9,Afrique Nord-Ouest,98997,59887,321,22116,16673
36789,ZZ,FRANCAIS DE L'ETRANGER,10,"Afrique Centre, Sud et Est",89859,46782,566,17008,25503


In [72]:
regions_and_departments

,code_reg,name_reg,code_dep,name_dep
0,84,Auvergne-Rhône-Alpes,1,Ain
1,32,Hauts-de-France,2,Aisne
2,84,Auvergne-Rhône-Alpes,3,Allier
3,93,Provence-Alpes-Côte d'Azur,4,Alpes-de-Haute-Provence
4,93,Provence-Alpes-Côte d'Azur,5,Hautes-Alpes
...,...,...,...,...
104,COM,Collectivités d'Outre-Mer,984,Terres australes et antarctiques françaises
105,COM,Collectivités d'Outre-Mer,986,Wallis et Futuna
106,COM,Collectivités d'Outre-Mer,987,Polynésie française
107,COM,Collectivités d'Outre-Mer,988,Nouvelle-Calédonie


In [73]:
referendum_and_areas

,Department code,Department name,Town code,Town name,Registered,Abstentions,Null,Choice A,Choice B,code_reg,name_reg,code_dep,name_dep
0,1,AIN,1,L'Abergement-Clémenciat,592,84,9,154,345,84,Auvergne-Rhône-Alpes,1,Ain
1,1,AIN,2,L'Abergement-de-Varey,215,36,5,66,108,84,Auvergne-Rhône-Alpes,1,Ain
2,1,AIN,4,Ambérieu-en-Bugey,8205,1698,126,2717,3664,84,Auvergne-Rhône-Alpes,1,Ain
3,1,AIN,5,Ambérieux-en-Dombes,1152,170,18,280,684,84,Auvergne-Rhône-Alpes,1,Ain
4,1,AIN,6,Ambléon,105,17,1,35,52,84,Auvergne-Rhône-Alpes,1,Ain
...,...,...,...,...,...,...,...,...,...,...,...,...,...
36560,95,VAL D'OISE,676,Villers-en-Arthies,382,64,5,87,226,11,Île-de-France,95,Val-d'Oise
36561,95,VAL D'OISE,678,Villiers-Adam,595,71,4,191,329,11,Île-de-France,95,Val-d'Oise
36562,95,VAL D'OISE,680,Villiers-le-Bel,13269,4083,224,5546,3416,11,Île-de-France,95,Val-d'Oise
36563,95,VAL D'OISE,682,Villiers-le-Sec,95,11,0,40,44,11,Île-de-France,95,Val-d'Oise
